# Predecir y ordenar 

### Autor: Federico Picado
### Fecha de última modificación: 15/11/2024
### Descripción:

Este script predice N modelos donde solo cambia la semilla y mergea los resultados segun la metrica que escoja.

## Parámetros

In [1]:
mes_test = 202106 # mes donde se predice

# nombre de la carpeta donde se guardaron los modelos
nombre_carpeta = "modelo_DQ_IPC_v2" # Cambiar

# nombre de la carpeta donde se guardan los resultados para cada modelo
dataset_folder="prediccion_DQ_IPC_v2"

# metricas, pueden ser promedio, mediana, max, bootstrap. Si eligen bootstrap, tambien elegir n_bootstrap
metrica_elegida="promedio"
n_bootstrap=None # cambiar 

## Input

In [2]:
# Ya en el input se ejecutan cosas
import pandas as pd
import numpy as np
import lightgbm as lgb
import os

/home/fedepicado/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/exp/'
predicciones_path = base_path + 'buckets/b1/predicciones/'
db_path = base_path + 'buckets/b1/db/'
dataset_path = base_path + 'buckets/b1/datasets/'
dataset_file = 'competencia_02_DQ_IPC_v2.parquet'
full_path = dataset_path + dataset_file


## Carpeta donde guardo voy a guardar las predicciones del modelo que entrene
nombre_carpeta="modelo_DQ_IPC_v2"

Esta parte podria estar en procesamiento pero creo que aca se va a poder modificar mejor en el caso que el modelo no sea el mismo

In [4]:
### El modelo que entre yo es un LGBM, creando la clase binaria y separe los datos para predecir
data = pd.read_parquet(dataset_path + dataset_file)
df=data.copy()

df['clase_peso'] = 1.0
df['clase_binaria'] = 0
df['clase_binaria'] = np.where(df['clase_ternaria'] == 'CONTINUA', 0, 1)

df.loc[df['clase_binaria'] == 1, 'clase_peso'] = 1.0001

test_data = df[df['foto_mes'] == mes_test]
X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [5]:
predicciones_path = base_path + 'buckets/b1/predicciones/'
#dentro de predicciones_path se va a crear una carpeta para guardar las predicciones de cada modelo segun la metrica que elija. 

# MAIN

In [6]:
#levantar todos los modelos que haya dentro de la carpeta correspondiente

def cargar_modelos_desde_carpeta(modelos_path, nombre_carpeta):
    """
    Carga todos los modelos de LightGBM guardados en archivos dentro de la carpeta especificada.
    
    Parámetros:
    - modelos_path: str, ruta base donde se encuentra la carpeta de modelos.
    - nombre_carpeta: str, el nombre de la subcarpeta donde se guardaron los modelos.
    
    Retorna:
    - modelos_cargados: diccionario con los modelos cargados, donde la clave es el nombre del archivo y el valor es el modelo.
    """
    # Construir la ruta completa de la carpeta de modelos
    ruta_carpeta = os.path.join(modelos_path, nombre_carpeta)
    
    modelos_cargados = {}

    # Verificar que la carpeta existe
    if not os.path.exists(ruta_carpeta):
        print(f"La carpeta {ruta_carpeta} no existe.")
        return modelos_cargados

    # Listar y cargar solo archivos que terminan en '.txt'
    archivos_modelo = [f for f in os.listdir(ruta_carpeta) if f.endswith('.txt')]
    for archivo in archivos_modelo:
        ruta_del_modelo = os.path.join(ruta_carpeta, archivo)
        modelo = lgb.Booster(model_file=ruta_del_modelo)
        modelos_cargados[archivo] = modelo
        print(f"Modelo cargado desde: {ruta_del_modelo}")
    
    return modelos_cargados

In [7]:
modelos_cargados = cargar_modelos_desde_carpeta(modelos_path, nombre_carpeta)

Modelo cargado desde: /home/fedepicado/buckets/b1/exp/modelo_DQ_1/modelo_DQ_1_65033.txt
Modelo cargado desde: /home/fedepicado/buckets/b1/exp/modelo_DQ_1/modelo_DQ_1_75011.txt


In [8]:
def predecir_y_ordenar_modelos(modelos_entrenados, X_test, metrica="promedio", n_bootstrap=0,
                               predicciones_path=predicciones_path, nombre_carpeta=nombre_carpeta):
    """
    Realiza predicciones con modelos entrenados de LightGBM y crea un DataFrame final
    con la probabilidad obtenida mediante una métrica seleccionada para cada cliente.
    También permite guardar el DataFrame final en un archivo CSV en una carpeta específica.
    
    Parámetros:
    - modelos_entrenados: dict, contiene los modelos entrenados con las semillas como nombres.
    - X_test: DataFrame, conjunto de test.
    - metrica: str, criterio de selección de la probabilidad final ("promedio", "mediana", "max" o "bootstrap").
    - n_bootstrap: int, número de muestras bootstrap para el ensamble (por defecto 0, sin bootstrap).
    - predicciones_path: str, ruta base donde se guardarán las predicciones.
    - nombre_carpeta: str, nombre de la subcarpeta donde se guardarán las predicciones.
    
    Retorna:
    - df_final: DataFrame con 'numero_de_cliente' y la probabilidad seleccionada ('Probabilidad') para cada cliente.
    """
    
    # Asegurarse de que 'numero_de_cliente' esté en X_test
    if 'numero_de_cliente' not in X_test.columns:
        raise ValueError("La columna 'numero_de_cliente' debe estar presente en X_test")
    
    # Obtener las características utilizadas por los modelos
    columnas_modelo = modelos_entrenados[list(modelos_entrenados.keys())[0]].feature_name()
    
    # Asegurarse de que X_test tenga las mismas columnas que los datos de entrenamiento
    X_test_modelo = X_test[columnas_modelo].copy()
    
    # Crear un DataFrame para almacenar todas las predicciones
    todas_predicciones = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente']})
    
    # Para cada modelo entrenado, hacemos predicciones
    for nombre_modelo, model in modelos_entrenados.items():
        try:
            # Hacer predicciones sobre el conjunto de test
            predicciones = model.predict(X_test_modelo)
            
            # Agregar las predicciones al DataFrame
            todas_predicciones[nombre_modelo] = predicciones
            
            print(f"Predicciones realizadas para {nombre_modelo}")
        
        except Exception as e:
            print(f"Error al procesar {nombre_modelo}: {str(e)}")
            continue
    
    # Seleccionar la probabilidad final según el parámetro 'metrica'
    if metrica == "promedio":
        todas_predicciones['Probabilidad'] = todas_predicciones.iloc[:, 1:].mean(axis=1)
    elif metrica == "mediana":
        todas_predicciones['Probabilidad'] = todas_predicciones.iloc[:, 1:].median(axis=1)
    elif metrica == "max":
        todas_predicciones['Probabilidad'] = todas_predicciones.iloc[:, 1:].max(axis=1)
    elif metrica == "bootstrap":
        if n_bootstrap <= 0:
            raise ValueError("Debe especificar un número positivo de muestras bootstrap para el ensamble.")
        else:
            # Realizar ensamble de bootstrap
            predicciones_bootstrap = []
            num_modelos = len(modelos_entrenados)
            nombres_modelos = list(modelos_entrenados.keys())
            for i in range(n_bootstrap):
                # Seleccionar modelos con reemplazo
                modelos_seleccionados = np.random.choice(nombres_modelos, size=num_modelos, replace=True)
                # Promediar las predicciones de los modelos seleccionados
                promedio_bootstrap = todas_predicciones[modelos_seleccionados].mean(axis=1)
                predicciones_bootstrap.append(promedio_bootstrap)
            # Promediar los resultados de las muestras bootstrap
            todas_predicciones['Probabilidad'] = pd.concat(predicciones_bootstrap, axis=1).mean(axis=1)
    else:
        raise ValueError("La métrica debe ser 'promedio', 'mediana', 'max' o 'bootstrap'")
    
    # Crear el DataFrame final con 'numero_de_cliente' y la probabilidad seleccionada
    df_final = todas_predicciones[['numero_de_cliente', 'Probabilidad']]
    
    # Ordenar el DataFrame final por 'Probabilidad' de forma descendente
    df_final = df_final.sort_values(by='Probabilidad', ascending=False).reset_index(drop=True)
    
    # Si se proporciona predicciones_path, guardar el archivo
    if predicciones_path is not None:
        # Crear la carpeta completa (predicciones_path + nombre_carpeta) si no existe
        full_path = os.path.join(predicciones_path, nombre_carpeta)
        os.makedirs(full_path, exist_ok=True)
        
        # Crear el nombre del archivo usando la métrica
        file_name = f"predicciones_{metrica}.csv"
        file_path = os.path.join(full_path, file_name)
        
        # Guardar el DataFrame en un archivo CSV
        df_final.to_csv(file_path, index=False)
        print(f"El DataFrame final ha sido guardado en: {file_path}")
    
    return df_final

In [9]:
#definir la ruta donde se guardan las predicciones y el nombre de la carpeta donde se van a guardar
predecir_y_ordenar_modelos(modelos_cargados, X_test, metrica=metrica_elegida, n_bootstrap=n_bootstrap)

Predicciones realizadas para modelo_DQ_1_65033.txt
Predicciones realizadas para modelo_DQ_1_75011.txt
El DataFrame final ha sido guardado en: /home/fedepicado/buckets/b1/predicciones/modelo_DQ_1/predicciones_promedio.csv


,numero_de_cliente,Probabilidad
0,560453687,9.528198e-01
1,1134986809,9.462462e-01
2,1224175879,9.305245e-01
3,1026240450,9.262970e-01
4,883803687,9.244647e-01
...,...,...
164871,914491324,1.618978e-08
164872,477105711,1.547344e-08
164873,486163687,1.444123e-08
164874,550904181,1.154428e-08
